# Pygenlib 

Pygenlib is a python utilities library for handling genomics data.
It is roughly structured into the following modules:

- [iterators](#Iterators): efficient iteration over large-scaled genomics datasets. Iterators keep track of the genomic region of the
  yielded data enabling their efficient integration with other genomics data
- [genemodel](#Transcriptome): python classes for modeling genomics (annotation) data. This includes a '[transcriptome](#Transcriptome)' implementation 
  that models gene/transcript annotations and many useful querying/annotation methods.
- [utils](#Utility-functions): general (low-level) utility functions for working with genomics datasets.


In [1]:
# set path and install missing libraries
import os, pathlib
from platform import python_version
PYGENLIB_SRC='/groups/ameres/Niko/workspace/pygenlib/' # pathlib.Path().cwd().parent # '/Users/niko.popitsch/git/pygenlib/' 
os.chdir(PYGENLIB_SRC)
# install libraries. better to use a venv here!
#!python -m pip install -r requirements.txt 
display(f"Running pygenlib on python {python_version()}. Using pygenlib code from {PYGENLIB_SRC}")

'Running pygenlib on python 3.9.12. Using pygenlib code from /groups/ameres/Niko/workspace/pygenlib/'

In [2]:
# load libs
from IPython.display import Markdown
from Bio.Seq import Seq
import matplotlib.pyplot as plt
from pygenlib.utils import *
from pygenlib.iterators import *
from pygenlib.genemodel import *
SEP=Markdown('---')
!pwd

/groups/ameres/Niko/workspace/pygenlib


## Genomic intervals
Genomic regions are referenced by *genomic intervals* (gi) in pygenlib:
- Genomic intervals (gi) in pygenlib are inclusive and 1-based. 
- GIs are implemented as frozen (pseudo immutable) dataclasses and can thus be used, e.g., as keys in a dict.
- GIs can be instantiated by passing chrom/start/stop coordinates or can be parsed form a string. 
- GIs can be stranded. Unstranded intervals are represented by setting strand to None (default) 
- Using None for each component of the coordinates is allowed to represent unbounded intervals. Examples:
  - gi('chr1') refers to the whole chromosome
  - gi('chr1', 100000) refers to the section of chromosome 1 from (and including) positon 100k on.
  - gi(start=100, end=200) refers to positions 100-200 (inclusive) on any chromosome 
- Points are represented by GIs with same start and end coordinate.
- Empty intervals are represented by GIs with start>end coordinates (experimental)

Chromosome order in a genome is represented by *ReferenceDict*s (which extend regular python dicts).
- ReferenceDicts can be used to properly sort genomic intervals (see example below)
- ReferenceDicts keep track of the available chromosomes and their length and are typically directly derived from genomic data files
- ReferenceDicts are used in pygenlib to check for compatibility of different genomic datasets

In [ ]:
# create a list of genomic intervals
locs=[gi('chr2', 1, 100), # chr2:1-100, unstranded
      gi.from_str('chr1:5-500 (+)'),  # parsed from string, strand='+'
      gi.from_str('chr3:50-120 (-)'), 
      gi.from_str('chr1:10-20 (-)'), 
      gi(None, None, 1000, '-'), # chromosome and start unbound, end=1000, strand='-'
      gi(None, 10, 1000, '-')] # chromosome unbound
display('Sorted by start coordinate but order of chromosomes is undefined:', sorted(locs), SEP)

# Note that chromosome group intervals and the order of intervals from different groups (chromosomes) is left undefined.
display(f"Is the greater-than comparison chr2:1-1>chr1:1-1 defined? {gi('chr2', 1, 1) > gi('chr1', 1, 1)}", SEP)

# To sort also by chromosome, you can use a reference dict which defined the chromosome order:
refdict=ReferenceDict({'chr1':None, 'chr2':None, 'chr3':None}, 'test', None)
display(refdict, SEP)
display('Properly sorted list:', sorted(locs, key=lambda x: (refdict.index(x.chromosome), x)))
# Unbounded intervals (chromosome=None) will always be at the beginning of the list

# Iterators

_pygenlib_ implements several *LocationIterator*s for efficient iteration over (parts of) genomics datasets while keeping track of the [genomic interval](#Genomic-intervals) each yielded item refers to. Most implemented iterators are based on respective [pysam](https://pysam.readthedocs.io/en/latest/api.html) classes but extend them with different filtering/access methods and add sanity checks to avoid common errors when combining different genomics datasets.

The primary goal of LocationIterators is to support the synchronization/integration of different genomics datasets. LocationIterators also support chunked I/O where feasible and not supported by the underlying (pysam) implementation.

LocationIterators can iterate whole datasets or specific sub-region (e.g., a certain chromosome or genomic region). Yielded data items are sorted by respective [genomic intervals](#Genomic-intervals) where the order of chromosomes is defined by a [ReferenceDict](#Genomic-intervals) that is derived directly from the iterated dataset (e.g., by querying it's tabix index). When multiple LocationIterators are integrated (e.g., by an AnnotationIterator), their ReferenceDicts are checked for compatibility.

Sometimes, genomics datasets from different providers used differing chromosome ids to refer to the same chromosomes which hinders their integration. 
Location iterators support chromosome aliasing as a quick solution for this: users can provide aliasing functions (e.g., for adding/removing 'chr' prefixes from 
chromosome ids) to dynamically change chromosome ids while keeping the underlying data files untouched.

Implemented LocationIterators include:
* [FastaIterator](#FastaIterator): iterates FASTA files per position or genomic window (e.g., Tiling/sliding windows).
* [GFF3Iterator](#GFF3Iterator): iterates GTF/GFF3 files and parses annotation data. Used for the [Transcriptome](#transcriptome) implementation
* [ReadIterator](#ReadIterator): iterates SAM/BAM files and implements various filter options. 
* [VcfIterator](#AnnotationIterator): iterates VCF files and can filter for samples
* [AnnotationIterator](#AnnotationIterator): annotates the items from one LocationIterator with items from others

and many more.
The following sections showcase some of the implemented iterators:

## FastaIterator
Iterates over a FASTA file but enables also tiling/sliding windows and padding.

In [ ]:
# FastaIterator support window/step size and padding
fasta_file='testdata/ACTB+SOX2.fa.gz'

# get 5mer sliding windows (step sie 2) with padding from a GRCh38 chr7 region around ACTB; show first and last 5 results
kmers=[s for _,s in FastaIterator(fasta_file, 'chr7', None, None, width=5, step=2, padding=True).take()]
display(kmers[:5]+["..."]+kmers[-5:])

## GFF3Iterator
Iterates over GFF3 files and yields dicts containing parsed annotation fields.

In [ ]:
# Show some data from a GFF3 file
GFF3Iterator('testdata/gencode.v39.ACTB+SOX2.gff3.gz').take()[:10]

# iterate whole GFF3 file and collect stats of contained feature_type annotations
Counter([info['feature_type'] for _,info in GFF3Iterator('testdata/gencode.v39.ACTB+SOX2.gff3.gz')])

## ReadIterator
Iterates SAM/BAM files and implements several filter options. Also keeps track of how many reads were filtered for what reason.
Can also yield mismatches wrt. the reference if MD tags are available (can be added, e.g., by [samtools calmd](http://www.htslib.org/doc/samtools-calmd.html) if not added by the mapper itself).


In [ ]:
# count reads per chromosome in a BAM using different filters
stats={x:Counter() for x in ['all', 'def', 'mq20', 'tag']}
with open_file_obj('testdata/small_example.bam') as bam:
    for chrom in get_reference_dict(bam):
        with ReadIterator(bam, chrom, flag_filter=0) as it: # all: no filtering
            it.take()
            stats['all'].update(it.stats)
        with ReadIterator(bam, chrom) as it: # def: default flag filter (as in IGV)
            it.take()
            stats['def'].update(it.stats)
        with ReadIterator(bam, chrom, min_mapping_quality=20) as it: # mq20: default flag filter, mapping quality >= 20
            it.take()
            stats['mq20'].update(it.stats)
        with ReadIterator(bam, chrom, tag_filters=[TagFilter('MD', ['100'])]) as it: # tag: default flag filter, MD tag must be '100'
            it.take()
            stats['tag'].update(it.stats)
display(stats)

In [ ]:
# count t/c mismatches per read in a BAM file, filtering for minimum base quality
tc_conv={}
for l,(r,mm) in ReadIterator('testdata/small_example.bam',report_mismatches=True, min_base_quality=10):
    if len(mm)>0:
        is_rev = not r.is_reverse if r.is_read2 else r.is_reverse
        refc = "A" if is_rev else "T"
        altc = "G" if is_rev else "C"
        mm_tc=[(off, pos1, ref, alt) for off, pos1, ref, alt in mm if ref==refc and alt==altc]
        if len(mm_tc) > 0:
            tc_conv[r.query_name, not r.is_read2]=mm_tc

# show the first 10 reads
display('10 reads and their mismatches', {k:tc_conv[k] for k in list(tc_conv.keys())[:10]}, SEP)

# count reads with more than one T/C conversion
display('Number of reads with ore than one T/C conversion', len({k:tc_conv[k] for k in list(tc_conv.keys()) if len(tc_conv[k])>1}),SEP)

# show MM of one such read
display('Example read with 2 mismatches:', tc_conv['HWI-ST466_135068617:8:2316:4251:54002', False])

## AnnotationIterator
Annotation iterators are meant to annotate the genomic intervals yielded by one location iterator with data from multiple others. 

In [ ]:
# Iterate over annotations in a genomic subregion of the flybase GTF and 
# list the genotypes of annotated SNPs for 3 samples that overlap with those annotations

gff_file='testdata/flybase.dmel-all-r6.51.sorted.gtf.gz' 
vcf_file='testdata/dmelanogaster_6_exported_20230523.vcf.gz'

with AnnotationIterator(GFF3Iterator(gff_file, '2L', 574299, 575733), 
                        VcfIterator(vcf_file, samples=['DGRP-208', 'DGRP-325', 'DGRP-721'])) as it:
    for loc, (v1,v2) in it.take()[:5]:
        display(f"{loc} {v1} {v2}", SEP)

In [ ]:
# Annotate all intervals in a BED file with sum of scores from a bedgraph file; supports bedgraph intervals >1bp: score contribution
# is calculated from the interval overlap
bed_file = 'testdata/test.bed.gz'
bedg_file = 'testdata/test.bedgraph.gz'

# overlap with bedgraph file, calculate overlap and sum scores
# NOTE this bedgraph file contains, e.g., an interval (1:7-10, 0.3)
with AnnotationIterator(BedIterator(bed_file), BedGraphIterator(bedg_file), labels=['scores']) as it:
    for loc, i in it:
        display(f"{i.anno.name}, sum:{sum([x.data*loc.overlap(x.location) for x in i.scores])}")

# Transcriptome
_Pygenlib_ provides a transcriptome implementation that enables convenient access/filtering of genomic annotations via python. 
Briefly,a (filtered) input GTF/GFF file is parsed and respective (frozen) dataclasses that derive from the generic Feature class are instantiated. 
The implementation keeps track of parent/child relationships (e.g., between genes, transcripts and their exons) and enables efficient iteration/querying and annotation of genomic annotations. Data fields and annotations can conveniently be accessed by `<feature>.<attribute>`. 

Note that the current implementation does not implement the full GFF3 format as specified [here](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md)
but currently rather supports various popular gff 'flavours' as published by encode, ensembl, ucsc and flybase.
Here is a brief description of the datamodel depicted below:
-   Model contains genes, transcripts and arbitrary sub-features (e.g., exons, intron, 3'/5'-UTRs, CDS) as defined
    in the GFF file. Note that intron annotations are calculated automatically from exon annotations unless configured otherwise. 
    Frozen dataclasses (derived from the 'Feature' class) are created for all parsed feature
    types automatically and users may configure which GTF/GFF attributes will be parsed and added to those dataclasses as fields.
    
-   A transcriptome maintains a dict mapping (frozen) features to dicts of arbitrary annotation values which supports the (incremental) annotation of annotation features.
    Annotation values can accesed like GFF-parsed fields via 'dot' notation: `<feature>.<attribute>`. 
    
-   The implementation exploits the hierarchical relationship between genes and their sub-features to optimize storage and computational requirements where posssible.        
    Genomic sequences, for example, can be loaded from a reference genome via load_sequences() but will only be directly stored in strings attached to gene feature.
    Sequences can then be accessed via transcriptome.get_sequence(). For sub-features (e.g., transcripts, exons, etc.) the respective sequence will be sliced from the 
    gene sequence on demand.  Note that despite being computed on demand, genomic sequences can still be accessed via `<feature>.sequence` which will call 
    transcriptome.get_sequence() with default parameters.
    
-   The get_sequence() method supports several modes:
    If `mode='rna'` is passed, the sequence is returned in 5'-3' orientation, i.e., they are reverse-complemented
    for minus-strand transcripts. The returned sequence will, however, still use the DNA alphabet (ACTG) to
    enable direct alignment/comparison with genomic sequences.
    if `mode='spliced'`, the spliced 5'-3' sequence will be returned.
    if `mode='translated'`, the spliced 5'-3' CDS sequence will be returned.
    
-   Genomic range queries via query() are supported by a combination of interval and linear search queries.
    A transcriptome object maintains one intervaltree per chromosome built from gene annotations.
    Overlap/envelop queries will first be applied to the respective intervaltree and the (typically small
    result sets) will then be filtered, e.g., for requested sub-feature types.
    
-   Transcriptome models can also be built from subsets of GFF/GTF files by filtering contained transcripts with a TranscriptFilter() that can restrict included chromosomes, genomic regions, transcript_ids, gene_types or transcripts with certain 'tag' values (as used in gencode).

-   The transcriptome build process is configured by a single JSON file/dict as shown below. 

The following cells demonstrate some use cases.

----

![transcriptome datamodel](pygenlib_transcriptome.png "Transcriptome datamodel")

## API Examples

### Basic access examples

In [ ]:
# First, let's create a transcriptome containing dmel chromosome 2L annotation from a flybase annotation file.
config = {
        'genome_fa': 'testdata/dmel_r6.36.fa.gz',                          # genome FASTA
        'annotation_gff': 'testdata/flybase.dmel-all-r6.51.sorted.gtf.gz', # Gene annotation GTF/GFF file
        'annotation_flavour': 'flybase',                                   # flavour of the cnnotation file.
        'transcript_filter': {                                             # a simple filter that will include only annotations from 2L
            'included_chrom': ['2L']
        },
        'load_sequences': True # load sequences from configured genome FASTA file
    }
t = Transcriptome(config)

In [ ]:
# Now, lets access some data:
display(f"Built {t}")                                                                              # show numbers of parsed genes/transcripts
display(f"Transcript types: {Counter([tx.transcript_type for tx in t.transcripts])}")              # count all transcript_type fields that were parsed from the flybase feature_types
display(f"Transcripts of gene 'cold': {[tx.transcript_id for tx in t.gene['cold'].transcript]}.")  # query gene by name and get its transcripts
display(f"Number of exons per cold tx: { {tx.transcript_id:len(tx.exon) for tx in t.gene['cold'].transcript} }") # get number of exons for all cold transcripts
display(f"So, the gene of transcript 'FBtr0310022' is {t.transcript['FBtr0310022'].parent.name}")  # Access transcript by id and show the name field of its gene (=parent)

In [ ]:
# Some more access examples
# genes can be accessed by name or gene id; their location
display(f"The gene {t.gene['cold'].name} is located on chromosome {t.gene['cold'].location.chromosome} at {t.gene['FBgn0031268'].location} ", SEP) 

# you can inspect the fields of a transcriptome feature with regular python methods, e.g. vars()
# Here we show, e.g., the structure of the 1st transcript
tx=t.gene['cold'].transcript[0]
display(vars(tx),SEP)

# fields but also dynamic annotations (see below) can be accessed by <feature>.<attribute/field> notation
# this also includes some calculated features such as location and rnk (exon/intron number)
display(f"transcript_id: {tx.feature_id}, location: {tx.location}, 1st exon rnk: {tx.exon[0].rnk}")


### Sequence access

In [ ]:
# Genomic sequences are another example for dynamically calculated fields. They must first be loaded for each gene annotation with load_sequences() 
# (which is done automatically if 'load_sequences': True is configured).
# They can be accessed by the t.get_sequence() method or by <feature>.sequence which calls get_sequence() with default params
display(f"Sequence of 1st FBtr0310022 exon: {t.transcript['FBtr0310022'].exon[0].sequence}",SEP)
display(f"Spliced sequence of FBtr0310022: {t.transcript['FBtr0310022'].spliced_sequence}",SEP)

# SJs can be displayed by calling get_sequence(mode='spliced', show_exon_boundaries=True)
display(f"Spliced sequence of FBtr0310022 showing SJ ('*'): {t.get_sequence(t.transcript['FBtr0310022'] ,mode='spliced', show_exon_boundaries=True)}",SEP)

# sequences will be reverse complemented for - strand transcripts if mode='rna'
# get random minustrand tx
tx=random.sample([tx for tx in t.transcripts if tx.strand=='-'], 1)[0]

# show sequence of 1st exon in dna and rna mode
display(f"tx {tx.transcript_id}, exon 1: dnaseq={t.get_sequence(tx.exon[0], mode='dna')}, rnaseq={t.get_sequence(tx.exon[0], mode='rna')}", SEP)

In [ ]:
# The transcriptome implementation also supports (annotated) CDS annotations. Users can directly access the translated sequence:
display(f"Translated sequence of FBtr0310022: {t.transcript['FBtr0310022'].translated_sequence}",SEP)

# align spliced and translated sequence
display(f"Alignment scores: {align_sequence(t.transcript['FBtr0310022'].translated_sequence, t.transcript['FBtr0310022'].spliced_sequence, print_alignment=True)}", SEP)

# translate with biopython
display(f"Amino acid sequence of FBtr0310022: {Seq(t.transcript['FBtr0310022'].translated_sequence).translate()}", SEP)

In [ ]:
# list all gene names where the kmer is found in one of the (spliced) transcripts
display(f"Genes containing kmer in one of their (spliced) tx: { { tx.parent.name for tx in t.transcripts if 'ACTGAGCTA' in tx.spliced_sequence }}", SEP)

# list all gene names where the kmer is found in one of its transcripts introns considering only long (>10kb) introns
display(f"Genes containing kmer in a long (>10kb) intron: { { tx.parent.name for tx in t.transcripts for intron in tx.intron if len(intron)>10000 and 'ACTGAGCTA' in intron.sequence}}", SEP)

### Range queries

In [ ]:
# Efficient range queries are implemented by intervaltrees:
display(f"Genes where an exon overlaps with 2L:20000-30000: { {ex.parent.parent.name for ex in t.query(gi.from_str('2L:20000-30000'), 'exon')}}", SEP) 

# Query all transcripts that overlap with the 'cold' gene (see 2L:574291-575734) and create a set with their gene names.
cold=t.gene['cold'] # note that each transcriptome feature is also a genomic interval and can be queried
display(f"Genes of all transcripts that overlap with gene 'cold': {set(tx.parent.name for tx in t.query(cold, 'transcript'))}", SEP) 

# report a coordinate-sorted list of genes in a 10kb window around cold:
display(f"Genes near 'cold': { [g.name for g in t.query(gi(cold.chromosome, cold.start-10000, cold.end+10000), 'gene')]}", SEP)

# envelop queries are also supported:
display(f"Genes contained in a +/- 10kb window around 'cold': { [g.name for g in t.query(gi(cold.chromosome, cold.start-10000, cold.end+10000), 'gene', envelop=True)]}")

### Special access methods

In [ ]:
# iterate genes and their up/downstream genes within a given max distance. Show only first 10 entries.
def get_name(x):
    return None if x is None else x.name
[(get_name(x),get_name(y),get_name(z)) for x, y, z in t.gene_triples(max_dist=10000)][:10] # Note that, e.g., for lncRNA:CR46254, there is no downstream gene within 10kb

In [ ]:
# Create a dict of genes and their unique 200bp 3'UTR intervals per tx (multiple intervals if spliced); show for 5 genes only
# note that for CG11374/FBtr0306541 2 intervals are reported as the 200bp length requires splicing
{g.name: {tx.transcript_id:calc_3end(tx) for tx in g.transcript} for g in t.genes[10:15]}

In [ ]:
# complex query examples
# find all minus strand genes with transcripts that have >=2 exons and at least one long (>10kb) intron
{tx.parent.name for tx in t.transcripts if tx.strand=='-' and len(tx.exon)==2 and any([len(i)>10000 for i in tx.intron])}

### Annotation of features

A transcriptome manages an `anno` dict that maps annotation features (e.g., an exon) to a dict containing arbitray annotations.
Annotations can be added manually or by using the `transcriptome.annotate()` method that implements a generic method to add feature-based annotations.
The following section demonstrates some use cases:

In [ ]:
# First, let's instantiate a small transcriptome that contains only two human genes (ACTB and SOX2) 
config = {
        'genome_fa': 'testdata/ACTB+SOX2.fa.gz',
        'genome_offsets': {'chr3': 181711825, 'chr7': 5526309},
        'annotation_gff': 'testdata/gencode.v39.ACTB+SOX2.gff3.gz',
        'annotation_flavour': 'gencode',
        '#transcript_filter': {
            'included_tids': ['ENST00000473257.3']
        },
        'load_sequences': True
    }
t=Transcriptome(config)

In [ ]:
# Now, lets annotate exons and introns with mappability scores that are read from a bedgraph file.
# We first create a method for calculating the mean mappability for an annotation based on the scores of
# all overlapping intervals from the bedgraph file.
def calc_mean(item, label='mappability'):
    loc, (anno, scores) = item
    anno[label]=sum([score*loc.overlap(sloc) for sloc,score in scores])/len(loc)
    return calc_mean

# now we can call the annotate method, tell it to get the scores from a bedgraph iterator and
# to annotate only exons + introns. Note that this test bedgraph file covers only ~half of ACTB,
# see IGV
t.annotate(iterators=BedGraphIterator('testdata/GRCh38.k24.umap.ACTB_ex1+2.bedgraph.gz'),
           fun_anno=calc_mean,
           feature_types=['exon', 'intron'])

# Now, lets show the mappability values for exons and introns of one random transcript 
rnd_tx = random.sample(t.gene['ACTB'].transcript, 1)[0]
display([(f.feature_id, f.mappability) for f in rnd_tx.exon])

# Note that it would be more efficient to annotate genes with mappability score arrays and then calculate mean/median mappability
# for each feature by slicing these arrays (as done, e.g., for sequences) but this is omited here for te sake of simplicity

In [ ]:
# access with default value
display(getattr(t.genes[0],'sequence',None))
display(f"gene has no mappability value: {getattr(t.genes[0],'mappability',None)}")
display(f"exon has mappability value: {getattr(t.genes[0].transcript[0].exon[0],'mappability',None)}")

## Pileup performance comparison

Here we compare our pileup method to pysam (which adds a lot of boilerplate + additional checks) and reach a 10X performance increase.

In [ ]:
reg=gi('1',22377202,22429853)

In [ ]:
%%timeit -n 1 -r 1
ac1=Counter()
with open_file_obj('testdata/small_example.bam') as bam:
    for l,v in FastPileupIterator(bam, reg.chromosome, range(reg.start, reg.end)):
        ac1[l.chromosome, l.start]=v
#display(ac1)

In [ ]:
%%timeit -n 1 -r 1
ac2=Counter()
with open_file_obj('testdata/small_example.bam') as bam:
    for pu in bam.pileup(contig=reg.chromosome, start=reg.start-1, stop=reg.end-1, flag_filter=DEFAULT_FLAG_FILTER, 
                         truncate=True,mark_ends=True,add_indels=True,min_base_quality=0,min_mapping_quality=0,
                         ignore_overlaps=False, ignore_orphans=False,
                         max_depth=100000):
        pos=(pu.reference_name, pu.reference_pos+1)
        ac2[pos]=Counter()
        for r in pu.pileups:
            if r.is_refskip:
                continue
            elif r.is_del:
                ac2[pos][None]+=1
            else:
                ac2[pos][r.alignment.query_sequence[r.query_position]]+=1
#display(ac2)

Finally, lets check whether results match (remove %%timeit first)

In [ ]:
for pos in ac1:
    if ac2[pos]==0:
        ac2[pos]=Counter()
if ac1==ac2:
    print("ALL GOOD")
else:
    for pos in ac1:
        if ac1[pos]!=ac2[pos]:
            print('err', pos, ac1[pos], ac2[pos])

# Complex transcriptome example

In this more complex example we will filter and visualize genomic features by read mappability 

Briefly, we will do the following:
- build a transcriptome of canonical protein coding genes from gencode annotations
- annotate all gene annotations with a large (3.1G) bedgraph file containing umap mappability values. Takes ~3h
- store the results in a pickle file so we can quickly reload later
- Query some if the data
- Visualize some of the data with matplotlib

In [3]:
config={
    'genome_fa': '/groups/ameres/Niko/ref/genomes/GRCh38/GCA_000001405.15_GRCh38_full_plus_hs38d1_analysis_set.fna',
    'annotation_gff': '/groups/ameres/Niko/ref/genomes/GRCh38/annotation/gencode.v39.annotation.sorted.gff3.gz',
    'annotation_flavour': 'gencode',
    'transcript_filter': {
        'included_tags': ['Ensembl_canonical'],
        'included_genetypes': ['protein_coding'],
        '#included_chrom': ['chr20']
    },
    'copied_fields': ['gene_type'],
    'drop_empty_genes': False,
    'load_sequences': False
}
# Build subset of human transcriptome (all protein coding genes, Ensembl canonical only), takes ~2min on CLIP
t=Transcriptome(config)
display(t)

WARN: Input list have differing order of shared elements ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrM', 'chrX', 'chrY']


Building transcriptome (Filtered (1 tags, 0 tids, 1 genetypes, 0 chroms, 0 regions).): 100%|██████████| 25/25 [01:05<00:00,  2.64s/it]
Build interval tree: 100%|██████████| 19964/19964 [00:00<00:00, 43558.44it/s]


Transcriptome with 19964 genes and 19964 tx

## Annotate genes with mappability values

In [4]:
#%%timeit -n 1 -r 1
import numpy as np
# Example code that calculates a numpy array containing mappability values for each genomic position of the passed genomic feature (anno).
# The respective values are then saved (pickled) to the given output file
# if the output file already exists, values will be loaded from there istead of recalculating them.

def anno_scores(item, label='mappability'):
    """ 
        Callback method for creating the mappability array by slicing all overlapping intervals.
        loc: genomic interval of the feature that is annotated
        anno: the transcriptome anno dict for this feature (so you can also access any other already existing annotations for this feature)
        scores: list of (loc, score) tuples containing all overlapping locations (loc) and their scores as read from the bedgraph file.
    """
    loc, (anno, scores) = item
    anno[label]=np.zeros(len(loc)) # create empty array
    for sloc,score in scores:
        anno[label][range(max(loc.start, sloc.start)-loc.start, min(loc.end,sloc.end)+1-loc.start)]=score # update scores

pkfile='/groups/ameres/Niko/ref/genomes/GRCh38/annotation/gencode.v39.k24.umap.genes.pk' # NOTE: different chr order: chr1, chr2, ...
bedgraph_file='/groups/ameres/Niko/ref/genomes/GRCh38/mappability/GRCh38.k24.umap.bedgraph.gz'

if os.path.isfile(pkfile): 
    t.load_annotations(pkfile) # load from pickle file if existing; use update=True to not lose any previously annotated values 
else:
    roi=None # annotate all; here you could provide a genomic interval to restrict what features should be annotated
    #annotate genes with mappability scores
    t.annotate(iterators=BedGraphIterator(bedgraph_file, region=roi), 
               region=roi,
               fun_anno=anno_scores,
               feature_types=['gene'])
    t.save_annotations(pkfile, keys={'mappability'}) # save only the mappability values to disk.

# show an estimate of the memory consumption (~10GB so it might make sense to calculate mean/median mappability per annotation instead of keeping all values)
print(f"Memory size of the create numpy arrays: {sum([a['mappability'].nbytes for a in t.anno.values() if 'mappability' in a]) / (1000*1000*1000) } GB")

Loading annotations from /groups/ameres/Niko/ref/genomes/GRCh38/annotation/gencode.v39.k24.umap.genes.pk
Memory size of the create numpy arrays: 10.964520664 GB


## Query some data

In [18]:
display(t.genes[0].transcript[0].get("mappability",  default_value=[], slice_from_parent=True))
display(t.slice_from_parent(t.genes[0].transcript[0], 'mappability', []))
display(t.genes[0].transcriptome.slice_from_parent(t.genes[0].transcript[0], 'mappability', []))

None

[]

array([0., 0., 0., ..., 0., 0., 0.])

array([0., 0., 0., ..., 0., 0., 0.])

In [7]:
# query example: 
# calculate mean exon mappability by slicing values from their parent genes and calculating the mean
# NOTE that the bedgraph file does not contain data for chrM and chrY (see tabix -l <file>), so for these we use the default value []
ex_map=[(ex, np.mean(ex.get('mappability', default_value=[], slice_from_parent=True))) for ex in t.__iter__(feature_types='exon')]
# get all exons with 0 mappability 
zero_map_ex=[ex for ex,mean_map in ex_map if mean_map==0]
# now show 10 random genes that contain at least 1 exon with zero mappability
display(f"There are {len(zero_map_ex)}/{len(ex_map)} exons with 0 mappability. Here are some example genes: { random.sample(list({ex.parent.parent.name for ex in zero_map_ex}), 10) }")

ValueError: Sample larger than population or is negative

## Plot some data 

In [ ]:
mean_map=[[np.mean(t.slice_from_parent(g, 'mappability', [])) for g in t.genes if g.chromosome==c] for c in t.merged_refdict]

In [ ]:
# fig, ax = plt.subplots()
# ax.set_xlabel('chromosome')
# ax.set_ylabel('mappability')
_=plt.boxplot(mean_map)
_=plt.xticks([x+1 for x in range(len(t.merged_refdict))], list(t.merged_refdict.keys()), rotation=90)
_=plt.suptitle("Gene region mappability per chromosome")
_=plt.title("No data for chrY/chrM")

In [ ]:
# quick check that there are really no wrong overlaps reported for chroms M+Y
def show_overlap(item):
    loc, (anno, scores) = item
    print(loc, scores)

bedgraph_file='/groups/ameres/Niko/ref/genomes/GRCh38/mappability/GRCh38.k24.umap.bedgraph.gz'
for chrom in ['chrY','chrM']:
    t.annotate(iterators=BedGraphIterator(bedgraph_file), 
               region=gi(chrom),
               fun_anno=show_overlap,
               feature_types=['gene'])

# Utility functions

_Pygenlib_ also contains a number of utility functions which might be useful for handling genomics data. Some of those functions are demonstrated in the following:

In [ ]:
# Gene symbols are updated regularly and mapping between different id schemas is cumbersome
# pygenlib implements an interface to MyGeneInfo for easy translation between ids and symbols 
# Example: we pass a mixed list of Ensembl and Entrez ids for mouse and human  actin beta:
geneid2symbol(['ENSMUSG00000029580', 60])

In [ ]:
# Here we use genenames.org data to convert lists containing partially old gene symbols to their current symbol
#gene_name_alias_file='/Users/niko.popitsch/Desktop/data/projects/Ameres/af2/colabfold_test/annotations/hgnc_complete_set.txt'
gene_name_alias_file='/groups/ameres/Niko/ref/genomes/GRCh38/annotation/20230206_hgnc_complete_set.txt'

aliases, current_symbols = read_alias_file(gene_name_alias_file)
# now lets translate some previous names of AADAC
norm_gn('DAC', current_symbols, aliases), norm_gn('CES5A1', current_symbols, aliases)

In [ ]:
# We also provide a wrapper around some useful Biopython methods, e.g., for sequence alignment
refseq='AAATTTCCCACTGAAATTTCCC'
score, startpos, endpos = align_sequence('ACTGactTTTC', refseq, print_alignment=True)
print(f"We found a match from {startpos}-{endpos} ('{'AAATTTCCCACTGAAATTTCCC'[startpos:endpos]}') with normalized alignment score {score}")

In [ ]:
# And utilities to work with Nanopore data (FAST5 files)
# You can, e.g., inspect the structure of such a file:
fast5_file='/groups/ameres/Niko/projects/Ameres/nanopore/data/NA12878_HG001/0_fast5/Bham_Run1/Bham_Run1.batch0_0.fast5' # testdata/FAT61995_a1291c8f_5.fast5
print_fast5_tree(fast5_file, show_attrs=False)

In [ ]:
# Or check the basecalling groups in this file
fast5_file='/groups/ameres/Niko/projects/Ameres/nanopore/data/nanocall_gfp/results/basecalled/wt/FAQ55498_b5ea166b_0.fast5'
get_bcgs(fast5_file)